## RAG-application
### Setup och Ladda Databas

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Vektordatabas och Embeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Prompting, LLM och för att extrahera text från LLM-output
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# ==========================================================
# 1. Ladda ENV och definiera sökvägar
# ==========================================================
load_dotenv()

BASE_DIR = Path(r"C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden")
DB_DIR = BASE_DIR / "data" / "03_vector_db" / "green_power_sweden_db" 

# ==========================================================
# 2. Ladda embeddingmodell
# ==========================================================
print("Laddar embedding-modell...")

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

# ==========================================================
# 3. Ladda vektordatabasen
# ==========================================================
print(f"Laddar databas från {DB_DIR}...")

# Försök ladda databasen
try:
    vectordb = Chroma(
        persist_directory=str(DB_DIR),
        embedding_function=embedding_model
    )
    print("Databas laddad!")
    # OBS: count() kan vara tungt om indexet är trasigt, men vi testar
    print("Antal chunks:", vectordb._collection.count())
    
except Exception as e:
    print(f"\nKRITISKT FEL VID LADDNING AV DATABAS: {e}")
    print("Detta beror troligen på versionsskillnader mellan Colab och din dator.")

Laddar embedding-modell...
Laddar databas från C:\Users\Dator\Documents\Data_Science\11_Examensarbete\green_power_sweden\data\03_vector_db\green_power_sweden_db...
Databas laddad!
Antal chunks: 165583


### Testa sökning (Utan LLM)

In [2]:
# ==========================================================
# 4. TESTA SÖKNING (utan LLM)
# ==========================================================
query = "Får man bygga solcellsparker på jordbruksmark?"
print(f"\nTestar retrieval för frågan: '{query}'")

docs = vectordb.similarity_search(query, k=3)

print("\n--- RESULTAT (UTAN LLM) ---")
for i, doc in enumerate(docs):
    print(f"\n--- Dokument {i + 1} ---")
    print("Källa:", doc.metadata.get("source", "Okänd"))
    print(doc.page_content[:300], "...")


Testar retrieval för frågan: 'Får man bygga solcellsparker på jordbruksmark?'

--- RESULTAT (UTAN LLM) ---

--- Dokument 1 ---
Källa: Samrådsunderlag_Hylinge_1.0(22895278) (0)_TMP.pdf
motverka den låga mullhalten i området. Detta leder till ett stort näringsläckage som
skulle undvikas med en solcellspark på platsen.
Parken kommer inte att påverka transporter eller tillgänglighet till att bruka närliggande
jordbruksmarker.
Jordbruksmarkens kvalitet kommer inte försämras till följd ...

--- Dokument 2 ---
Källa: Samrådsanmälan_Hylinge_1.0(22417535) (0)_TMP.pdf
motverka den låga mullhalten i området. Detta leder till ett stort näringsläckage som
skulle undvikas med en solcellspark på platsen.
Parken kommer inte att påverka transporter eller tillgänglighet till att bruka närliggande
jordbruksmarker.
Jordbruksmarkens kvalitet kommer inte försämras till följd ...

--- Dokument 3 ---
Källa: MMD_2024-10-31_M2145-24_Funbo I_ok lokalisering.pdf
solcellspark kommer att uppnå den lönsamhet som kr

### Koppla in Gemini (Chatt)

In [4]:
# ==========================================================
# 5. Skapa Gemini LLM
# ==========================================================
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.3
)


# ==========================================================
# 6. Promptmall (ny LCEL-kompatibel)
# ==========================================================
prompt = ChatPromptTemplate.from_template("""
Du är *Solaris Legal*, expert på svensk juridik för solcellsparker.
Besvara frågan baserat på följande kontext.

Om svaret inte finns i kontexten: svara 
"Jag hittar inte svaret i de dokument jag har tillgång till."

Hitta inte på fakta. Hänvisa till källor om möjligt.

KONTEKST:
{context}

FRÅGA:
{question}

SVAR:
""")

retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# ==========================================================
# 7. Skapa RAG-kedja (NYA SÄTTET — LCEL)
# ==========================================================
def run_rag(question):
    context_docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in context_docs)

    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"context": context, "question": question})


# ==========================================================
# 8. Testa RAG med Gemini
# ==========================================================
print("\n\n🚀 Testar RAG med Gemini:\n")

answer = run_rag("Får man bygga solcellsparker på jordbruksmark?")
print(answer)



🚀 Testar RAG med Gemini:

Ja, det är möjligt att bygga solcellsparker på jordbruksmark, men det innebär vissa överväganden och inskränkningar.

Enligt Mark- och miljödomstolen vid Nacka tingsrätts avgörande från 2023-04-28 medgavs en solcellspark på jordbruksmark i Norrtälje kommun, trots att den låg inom riksintresse för kulturmiljö (Sida 34 av 4854).

Dock konstaterar domstolen att den aktuella marken utgör "brukningsvärd jordbruksmark" i den mening som avses i 3 kap. 4 § miljöbalken, vilken betonar ett långsiktigt hushållningsperspektiv där jordbruk är av nationell betydelse. Solcellsparkens konstruktion och livslängd (cirka 45 år) innebär att markanvändningen för jordbruk kommer att inskränkas. Det kan bli möjligt för exempelvis fårbete, men marken går inte att odla i mer konventionell bemärkelse under parkens livslängd (Sida 34 av 4854).

Positiva aspekter som lyfts fram är att jordbruksmarkens kvalitet inte förväntas försämras till följd av solcellsparken, och det kommer att va